The UK Food Standards Agency evaluates various establishments across the United Kingdom, and gives them a food hygiene rating. You've been contracted by the editors of a food magazine, Eat Safe, Love, to evaluate some of the ratings data in order to help their journalists and food critics decide where to focus future articles.

In [ ]:
#Import dependecies 
import pymongo 
import pymongo import MongoClient 
from pymongo import pprint 
import pandas as pd
import matplotlib.pyplot as plt

#Create an instance of MongoClient
mongo = MongoClient(port=27017)

#Assign the 'uk_food' database to a variable name
db = mongo['uk_food']

In [ ]:
#Review the collections in the database
#assign the collection to a variable
establishments = db['establishments']



The magazine editors have some requested modifications for the database before you can perform any queries or analysis for them. Make the following changes to the establishments collection:

An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following information to the database:

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
pprint(db.list_collection_names())

In [ ]:
# assign the collection to a variable
est = db['est']

In [ ]:
pprint(db.est.find_one())

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
print(f"There are {est.count_documents(query)} establishments with a hygiene score of 20")
# Display the first document in the results using pprint
#hyg = est(query)
pprint(est.find_one(query))

In [ ]:
# Convert the result to a Pandas DataFrame
hygdf = pd.DataFrame(est.find(query))
# Display the number of rows in the DataFrame
print("The number of rows is", len(hygdf))
# Display the first 10 rows of the DataFrame
hygdf.head(10)# Convert the result to a Pandas DataFrame
hygdf = pd.DataFrame(est.find(query))
# Display the number of rows in the DataFrame
print("The number of rows is", len(hygdf))
# Display the first 10 rows of the DataFrame
hygdf.head(10)

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName': {'$regex': 'London'}
print(f"There are {est.count_documents(query2)} establishments in London with a rating of 4 or higher")
# Display the first document in the results using pprint
pprint(est.find_one(query2))
#londonest = est.find(query2)
#ratelondon = londonest.drop({'RatingValue': {'$lt': '4'}})
#pprint(est.find_one(ratelondon))

In [ ]:
# Convert the result to a Pandas DataFrame
londondf = pd.DataFrame(est.find(query2))
# Display the number of rows in the DataFrame
print(f"The numbers of rows in the Dataframe is {len(londondf)}")
# Display the first 10 rows of the DataFrame
londondf.head(10)

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000


query = {'geocode.latitude': {'$gt': latitude - degree_search, '$lte': latitude + degree_search},
         'geocode.longitude': {'$gt': longitude - degree_search, '$lte': longitude + degree_search},
         'RatingValue': 5
        }
sort =  [('scores.Hygiene', 1)]

# Print the results
result = list(est.find(query).sort(sort).limit(5))
pprint(result)

In [ ]:
# Convert result to Pandas DataFrame
fivestar = pd.DataFrame(result)
fivestar.head()

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
matchpipe = {'$match': {'scores.Hygiene': 0}}
# 2. Groups the matches by Local Authority
grouppipe = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}
# 3. Sorts the matches from highest to lowest
sortpipe = {'$sort': {'count': -1}}

pipeline = [matchpipe, grouppipe, sortpipe]
resultpipe = list(est.aggregate(pipeline))
# Print the number of documents in the result
print(f"Number of locations with est. that have a Hygiene score of 0 is {len(resultpipe)}")
# Print the first 10 results
pprint(resultpipe[:10])